# Build a Rule-based Sentiment Classifier

This demo is a notebook with references from [CMU CS11-711 Advanced NLP](http://phontron.com/class/anlp2024/),  that trains a sentiment classifier based on data. Specifically, it uses a bag-of-words to extract features, and the structured perceptron algorithm to train the classifier.

It will take in a text `X` and return a `label` of \"1\" if the sentiment of the text is positive, \"-1\" if the sentiment of the text is negative, and \"0\" if the sentiment of the text is neutral. You can test the accuracy of your classifier on the [Stanford Sentiment Treebank](http://nlp.stanford.edu/sentiment/index.html) by running the notebook all the way to end.

In [ ]:
import random
import tqdm
!wget https://github.com/neubig/anlp-code/raw/refs/heads/main/data/sst-sentiment-text-threeclass/train.txt
!wget https://github.com/neubig/anlp-code/raw/refs/heads/main/data/sst-sentiment-text-threeclass/dev.txt

--2025-01-01 14:15:51--  https://github.com/neubig/anlp-code/raw/refs/heads/main/data/sst-sentiment-text-threeclass/train.txt
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/neubig/anlp-code/refs/heads/main/data/sst-sentiment-text-threeclass/train.txt [following]
--2025-01-01 14:15:52--  https://raw.githubusercontent.com/neubig/anlp-code/refs/heads/main/data/sst-sentiment-text-threeclass/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 940304 (918K) [text/plain]
Saving to: ‘train.txt’

train.txt           100%[===================>] 918.27K  --.-KB/s    in 0.009s  

2025-01-01 14:15:52 (

## Feature Extraction

Feature extraction code, how do we get the features we use in training? By default we just use every word.

In [ ]:
def extract_features(x: str) -> dict[str, float]:
    features = {}
    x_split = x.split(' ')
    for x in x_split:
        features[x] = features.get(x, 0) + 1.0
    return features

Also, initialize the feature weights to zero.

In [ ]:
feature_weights = {}

## Data Reading

Read in the data from the training and dev (or finally test) sets

In [ ]:
def read_xy_data(filename: str) -> tuple[list[str], list[int]]:
    x_data = []
    y_data = []
    with open(filename, 'r') as f:
        for line in f:
            label, text = line.strip().split(' ||| ')
            x_data.append(text)
            y_data.append(int(label))
    return x_data, y_data

In [ ]:
x_train, y_train = read_xy_data('train.txt')
x_dev, y_dev = read_xy_data('dev.txt')

In [ ]:
print(x_train[0])
print(y_train[0])

The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal .
1


## Inference Code

How we run the classifier.

In [ ]:
def run_classifier(features: dict[str, float]) -> int:
    score = 0
    for feat_name, feat_value in features.items():
        score = score + feat_value * feature_weights.get(feat_name, 0)
    if score > 0:
        return 1
    elif score < 0:
        return -1
    else:
        return 0

## Training Code

Learn the weights of the classifier.

In [ ]:
NUM_EPOCHS = 5
for epoch in range(1, NUM_EPOCHS+1):
    # Shuffle the order of the data
    data_ids = list(range(len(x_train)))
    random.shuffle(data_ids)
    # Run over all data points
    for data_id in tqdm.tqdm(data_ids, desc=f'Epoch {epoch}'):
        x = x_train[data_id]
        y = y_train[data_id]
        # We will skip neutral examples
        if y == 0:
            continue
        # Make a prediction
        features = extract_features(x)
        predicted_y = run_classifier(features)
        # Update the weights if the prediction is wrong
        if predicted_y != y:
            for feature in features:
                feature_weights[feature] = feature_weights.get(feature, 0) + y * features[feature]

Epoch 5: 100%|██████████| 8544/8544 [00:00<00:00, 90734.50it/s]


In [ ]:
print(f"Total features {len(feature_weights.keys())}")
for i,(k,v) in enumerate(feature_weights.items()):
  print(f"{k}\t\t{v}")
  if i>20:
    break

Total features 13658
It		3.0
is		2.0
different		5.0
from		1.0
others		3.0
in		-1.0
its		-2.0
genre		9.0
that		4.0
it		2.0
does		-1.0
not		-3.0
rely		2.0
on		2.0
dumb		-7.0
gags		-3.0
,		1.0
anatomical		4.0
humor		-6.0
or		-4.0
character		1.0
cliches		-4.0


## Evaluation Code

How we evaluate the classifier:

In [ ]:
def calculate_accuracy(x_data: list[str], y_data: list[int]) -> float:
    total_number = 0
    correct_number = 0
    for x, y in zip(x_data, y_data):
        y_pred = run_classifier(extract_features(x))
        total_number += 1
        if y == y_pred:
            correct_number += 1
    return correct_number / float(total_number)

In [ ]:
label_count = {}
for y in y_dev:
    if y not in label_count:
        label_count[y] = 0
    label_count[y] += 1
print(label_count)

{1: 444, 0: 229, -1: 428}


In [ ]:
train_accuracy = calculate_accuracy(x_train, y_train)
test_accuracy = calculate_accuracy(x_dev, y_dev)
print(f'Train accuracy: {train_accuracy}')
print(f'Dev/test accuracy: {test_accuracy}')

Train accuracy: 0.7833567415730337
Dev/test accuracy: 0.584014532243415


## Error Analysis

An important part of improving any system is figuring out where it goes wrong. The following two functions allow you to randomly observe some mistaken examples, which may help you improve the classifier. Feel free to write more sophisticated methods for error analysis as well.

In [ ]:
def find_errors(x_data, y_data):
    error_ids = []
    y_preds = []
    for i, (x, y) in enumerate(zip(x_data, y_data)):
        y_preds.append(run_classifier(extract_features(x)))
        if y != y_preds[-1]:
            error_ids.append(i)
    for _ in range(5):
        my_id = random.choice(error_ids)
        x, y, y_pred = x_data[my_id], y_data[my_id], y_preds[my_id]
        print(f'{x}\ntrue label: {y}\npredicted label: {y_pred}\n')

In [ ]:
find_errors(x_dev, y_dev)

A thinly veiled look at different aspects of Chinese life clashing with each other .
true label: 0
predicted label: 1

The movie achieves as great an impact by keeping these thoughts hidden as ... -LRB- Quills -RRB- did by showing them .
true label: 1
predicted label: 0

Scorsese does n't give us a character worth giving a damn about .
true label: -1
predicted label: 1

Even film silliness needs a little gravity , beyond good hair and humping .
true label: 0
predicted label: -1

It 's difficult to imagine the process that produced such a script , but here 's guessing that spray cheese and underarm noises played a crucial role .
true label: -1
predicted label: 1

